In [1]:
import sys
sys.path.append('../')

from rocket_rag.utils import *
from rocket_rag.agent import RagAgent

In [2]:
load = '40kg'

In [3]:
if_files_dict = parse_files(main_directory=INFERENCE_DIR)
if_ts_files = if_files_dict[load]

In [4]:
# np.random.seed(37)
rand_idx = np.random.randint(0, len(if_ts_files))
if_ts_filename = if_ts_files[rand_idx]
print(f'Randomly selected file: {if_ts_filename}')

if_rocket_feature = fit_transform(ts_filename=[if_ts_filename],
                                  field='current',
                                  smooth=True,
                                  smooth_ws=15,
                                  tolist=False,
                                  verbo=True)

2024-05-29 22:30:22.712 | DEBUG    | rocket_rag.utils:fit_transform:149 - Extract the time series data points
2024-05-29 22:30:22.719 | INFO     | rocket_rag.utils:fit:125 - time series extracted SUCCESSFULLY.
2024-05-29 22:30:22.719 | DEBUG    | rocket_rag.utils:fit_transform:161 - Rocket transforming...


Randomly selected file: ../data/inference/40kg\spalling8\spalling8_40_5_3.csv


In [5]:
agent = RagAgent(vs_dir=f'../store/nodes_{load}.pkl')

2024-05-29 22:30:28.215 | DEBUG    | rocket_rag.node_indexing:load_node_indexing:98 - Loading all nodes...
2024-05-29 22:30:28.760 | INFO     | rocket_rag.node_indexing:load_node_indexing:102 - All nodes are loaded.
2024-05-29 22:30:29.145 | INFO     | rocket_rag.agent:_init_openai_client:67 - OpenAI API key found. OpenAI client Initialized.
2024-05-29 22:30:29.146 | INFO     | rocket_rag.agent:__init__:59 - RAG agent initialized.


In [6]:
rocket_feature = agent.get_rocket_feature(if_ts_filename)
print(f'Rocket feature shape: {rocket_feature.shape}')
fault_prediction, score = agent.get_fault_prediction()
print(f'Fault prediction: {fault_prediction}, Score: {score}')

2024-05-29 22:30:29.160 | DEBUG    | rocket_rag.utils:fit_transform:149 - Extract the time series data points
2024-05-29 22:30:29.173 | INFO     | rocket_rag.utils:fit:125 - time series extracted SUCCESSFULLY.
2024-05-29 22:30:29.174 | DEBUG    | rocket_rag.utils:fit_transform:161 - Rocket transforming...


Rocket feature shape: (1, 20000)
Fault prediction: ['normal_40_8_4'], Score: 1.0


In [7]:
agent.generate_fault_diagnosis_statement()
print(agent.fault_diagnosis_json)

{'fault_type': 'normal', 'degradation_level': '', 'retrieval_result(s)': 'normal_40_8_4', 'score': '1.0', 'distances': [], 'description': 'The actuator operates under standard conditions without any faults. This state serves as a baseline for comparing the performance under fault conditions.'}


In [8]:
agent.refine_fault_diagnosis_statement()
print(agent.refined_diagnosis_json)

{'fault_type': 'normal', 'degradation_level': '', 'refinement': 'No obvious fault detection'}


In [9]:
agent.generate_multi_queries()
print(agent.generated_queries)

['linear actuator maintenance best practices', 'linear actuator performance optimization techniques', 'how to ensure long lifespan for linear actuators', 'linear actuator routine inspection checklist', 'preventative maintenance for linear actuators']


In [10]:
print(agent.tools.get_available_tools())

{'call_google': <function Tools.call_google at 0x00000279830C65C0>, 'save_to_file': <function Tools.save_to_file at 0x00000279830C6660>, 'get_available_tools': <function Tools.get_available_tools at 0x00000279830C6700>}


In [11]:
agent.call_google_search()
# print(agent.query_answers)

0it [00:00, ?it/s]

2024-05-29 22:30:44.045 | INFO     | rocket_rag.agent:call_google_search:283 - Calling Google search API for query 1/5
2024-05-29 22:31:01.041 | INFO     | rocket_rag.agent:call_google_search:283 - Calling Google search API for query 2/5


KeyError: 'snippet'

In [ ]:
await agent.gather_query_answers()

2024-05-29 22:27:50.835 | INFO     | rocket_rag.agent:gather_query_answers:352 - Start to generate the summerization for 2 nodes...
2024-05-29 22:28:19.894 | INFO     | rocket_rag.agent:gather_query_answers:361 - Combined into 2 responses, keep combining
2024-05-29 22:28:19.896 | INFO     | rocket_rag.agent:gather_query_answers:352 - Start to generate the summerization for 1 nodes...
2024-05-29 22:28:41.286 | INFO     | rocket_rag.agent:gather_query_answers:358 - Combined all responses to one. Done


"The comprehensive search results provide an in-depth overview of approaches to repair spalling damage in linear actuators and ballscrew mechanisms, alongside recovery techniques for spalling in ball screw mechanisms. These findings emphasize the necessity of understanding the underlying causes of such mechanical issues, employing specific repair strategies, and considering the structural implications of the damage.\n\nFor linear actuators, repair strategies vary from complete rebuilds to address different forms of damage, including spalling, to the use of additive manufacturing for part repairs. The nature of spalling is explored through comparisons with concrete and asphaltic surfaces, offering a wide perspective on repair methodologies. This includes troubleshooting advice for related damage, such as flaking in bearings and linear guides, and highlights the importance of thorough repairs and structural considerations, especially when dealing with severe damage that may affect system

In [ ]:
agent.parse_output_contents()

KeyError: 'refine_text'

In [ ]:
agent.save_output_contents(file_name='fault-diagnosis')